In [ ]:
import pandas
import pickle

import pandas as pd
import seaborn as sns
import platform
import re
from tqdm import tqdm

In [ ]:
YEWNO_DATA_PATH = '/Users/khoanguyen/Workspace/dataset/Yewno/ms_yewno_2020.pickle'
EDF_DATA_PATH = '/Users/khoanguyen/Workspace/dataset/edf_msft/MSFT_Jul2019_2020_linebreak'
FILTERED_YEWNO_PATH = '/Users/khoanguyen/Workspace/dataset/Yewno/filtered_yewno.pickle'
PERIOD_DATA_PATH = '/Users/khoanguyen/Workspace/dataset/edf_msft/'

monthly_file =  ['2019-07-01', '2019-08-01', '2019-09-01', '2019-10-01', '2019-11-01',
                 '2019-12-01', '2020-01-01', '2020-02-01', '2020-03-01', '2020-04-01',
                 '2020-05-01', '2020-06-01', '2020-07-01']

# Windows path
if platform.system() == 'Windows':
    YEWNO_DATA_PATH = 'K:\\Lbpam\\DG_Gestion_Quant\\GERANT\\Khoa\\yewno_ms_2020.pickle'
    EDF_DATA_PATH = 'K:\\Lbpam\\DG_Gestion_Quant\\GERANT\\Khoa\\Data\\MSFT\\MSFT_Jul2019_2020_linebreak'
    FILTERED_YEWNO_PATH = 'K:\\Lbpam\\DG_Gestion_Quant\\GERANT\\Khoa\\Data\\filtered_yewno.pickle'
    PERIOD_DATA_PATH = 'K:\\Lbpam\\DG_Gestion_Quant\\GERANT\\Khoa\\Data\\MSFT\\'

In [ ]:
with open(YEWNO_DATA_PATH, 'rb') as f:
    yewno_data = pickle.load(f)

In [ ]:
with open(EDF_DATA_PATH, 'rb') as f:
    edf_data = pickle.load(f)

In [ ]:
concept_count = []
concept_list = yewno_data['Concept'].tolist()

In [ ]:
for concept in tqdm(concept_list):
    # take care of string that contains quantifier, which can cause wonky interaction with re.findall
    if any(x in concept for x in ['+', '#', '.']):
        concept = re.escape(concept)
    # \b is word boundary, which doesn't work with string like 'C++' where there's no word boundary after ++
    re_string = r'(?:^|(?<=\s))' + concept + r'(?=\s|$)'

    # used re.IGNORECASE so that there's no need to convert case for string and text
    concept_per_doc = edf_data['text'].apply(lambda x: len(re.findall(re_string, x, flags=re.IGNORECASE)))
    concept_count.append(concept_per_doc.sum())

In [ ]:
yewno_data['edf_count'] = concept_count

In [ ]:
filtering_results = yewno_data[yewno_data['edf_count'] >= 10]

In [ ]:
filtering_results = filtering_results[(filtering_results['Pureplay'] > 0) &
                                      (filtering_results['Contribution'] > 0)]

In [ ]:
with open(FILTERED_YEWNO_PATH, 'wb') as f:
    pickle.dump(filtering_results, f)

In [ ]:
with open(FILTERED_YEWNO_PATH, 'rb') as f:
    filter_yewno_data = pickle.load(f)

In [ ]:
text_df = []

for data in monthly_file:
    with open(PERIOD_DATA_PATH + data, 'rb') as f:
        df = pickle.load(f)
        text_df.append(df)

In [ ]:
concept_list = filter_yewno_data['Concept'].tolist()

word_count_df = pd.DataFrame(index=monthly_file)
period_count = []
word_count = [[] for i in range(len(concept_list))]

for df in tqdm(text_df):
    for i in range(len(concept_list)):
        concept = concept_list[i]
        re_string = r'(?:^|(?<=\s))' + concept + r'(?=\s|$)'
        kw_per_doc = df['text'].str.count(re_string, flags=re.IGNORECASE)
        word_count[i].append(kw_per_doc.sum())

for i in range(len(concept_list)):
    word_count_df[concept_list[i]] = word_count[i]
word_count_df.fillna(0, inplace=True)

In [ ]:
# dask testing
import dask.dataframe as dd

dask_dataset = []
for df in text_df:
    ddf = dd.from_pandas(df, npartitions=2)
    dask_dataset.append(ddf)


In [ ]:
word_count_df = pd.DataFrame(index=monthly_file)
period_count = []
word_count = [[] for i in range(len(concept_list))]

for ddf in tqdm(dask_dataset):
    for i in range(len(concept_list)):
        concept = concept_list[i]
        re_string = r'(?:^|(?<=\s))' + concept + r'(?=\s|$)'
        findall_doc = ddf['text'].str.count(re_string, flags=re.IGNORECASE)
        word_count[i].append(findall_doc.sum().compute())

for i in range(len(concept_list)):
    word_count_df[concept_list[i]] = word_count[i]
word_count_df.fillna(0, inplace=True)